<a href="https://colab.research.google.com/github/Surya2000PratapSingh/AwesomeProject/blob/main/Blackcoffer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('conll2002')

In [11]:
import requests
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk import FreqDist

In [4]:
import pandas as pd

input_data = pd.read_excel('/content/Input.xlsx')
urls = input_data['URL'].tolist()
url_ids = input_data['URL_ID'].tolist()


In [ ]:
urls

In [ ]:
url_ids

In [5]:
import requests
from bs4 import BeautifulSoup

def extract_article_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    article_title = soup.find('title').text.strip()
    article_text = ''
    paragraphs = soup.find_all('p')
    for paragraph in paragraphs:
        article_text += paragraph.text.strip() + ' '

    return article_title, article_text

articles_data = []
for url, url_id in zip(urls, url_ids):
    title, text = extract_article_text(url)
    articles_data.append({'URL_ID': url_id, 'Title': title, 'Text': text})


In [12]:
from textblob import TextBlob
import re

def calculate_text_metrics(text):
    blob = TextBlob(text)

    positive_score = len([w for w in blob.words if TextBlob(w).sentiment.polarity > 0])
    negative_score = len([w for w in blob.words if TextBlob(w).sentiment.polarity < 0])
    polarity_score = blob.sentiment.polarity
    subjectivity_score = blob.sentiment.subjectivity
    sentences = re.split(r'[.!?]', text)
    avg_sentence_length = sum(len(sentence.split()) for sentence in sentences) / len(sentences)
    complex_word_count = len([w for w in blob.words if syllable_count(w) > 2])
    percentage_complex_words = (complex_word_count / len(blob.words)) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_words_per_sentence = len(blob.words) / len(sentences)
    word_count = len(blob.words)
    syllables_per_word = sum(syllable_count(w) for w in blob.words) / word_count
    personal_pronouns = len([w for w in blob.words if w.lower() in ['i', 'me', 'my', 'mine', 'myself', 'you', 'your', 'yours', 'yourself', 'we', 'us', 'our', 'ours', 'ourselves', 'yourselves', 'they', 'them', 'their', 'theirs', 'themselves']])
    avg_word_length = sum(len(w) for w in blob.words) / word_count

    return positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, percentage_complex_words, fog_index, avg_words_per_sentence, complex_word_count, word_count, syllables_per_word, personal_pronouns, avg_word_length


In [15]:
def syllable_count(word):
    word = word.lower()
    if word in ['i', 'a', 'as', 'at', 'e', 'o', 'u', 'y']:
        return 1
    else:
        return len(re.findall(r'[aeiouy]+', word))

metrics_data = []
for url, article in zip(urls, articles_data):
    positive_score, negative_score, polarity_score, subjectivity_score, avg_sentence_length, percentage_complex_words, fog_index, avg_words_per_sentence, complex_word_count, word_count, syllables_per_word, personal_pronouns, avg_word_length = calculate_text_metrics(article['Text'])
    metrics_data.append({
        'URL_ID': article['URL_ID'],
        'URL': url,
        'Positive_Score': positive_score,
        'Negative_Score': negative_score,
        'Polarity_Score': polarity_score,
        'Subjectivity_Score': subjectivity_score,
        'Avg_Sentence_Length': avg_sentence_length,
        'Percentage_of_Complex_Words': percentage_complex_words,
        'FOG_Index': fog_index,
        'Avg_Num_Words_Per_Sentence': avg_words_per_sentence,
        'Complex_Word_Count': complex_word_count,
        'Word_Count': word_count,
        'Syllables_Per_Word': syllables_per_word,
        'Personal_Pronouns': personal_pronouns,
        'Avg_Word_Length': avg_word_length
    })

metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_excel('output.xlsx', index=False)